<a href="https://colab.research.google.com/github/shirinrafia/https-github.com-RishubHighIQ-InstaBase_python_trainer/blob/main/Python_Trainer_Notebook_shirin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
my_set = {1,2,3,4,5}
your_set ={4,5,6,7,8,9,10}

print(my_set.union(your_set))


{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}


In [44]:
my_set = {1,2,3,4,5}
your_set ={4,5,6,7,8,9,10}

print(my_set.intersection(your_set))


{4, 5}


In [43]:
my_set = {1,2,3,4,5}
your_set ={4,5,6,7,8,9,10}

print(my_set.difference(your_set))
print(my_set)

{1, 2, 3}
{1, 2, 3, 4, 5}


In [42]:
my_set = {1,2,3,4,5}
your_set ={4,5,6,7,8,9,10}

print(my_set.discard(5))
print(my_set)

None
{1, 2, 3, 4}


In [39]:
my_set = {1,2,3,4,5}
your_set ={4,5,6,7,8,9,10}

print(my_set.difference(your_set))

{1, 2, 3}


In [38]:
my_set = {1,2,3,4,5}
my_set.add(100)
my_set.add(2)
print(my_set)

{1, 2, 3, 4, 5, 100}


In [37]:
my_tuple = (1, 2, 3, 4, 5)
new_tuple = my_tuple[1:4]
print(new_tuple)

(2, 3, 4)


In [34]:
user = {
    'basket' : [1,2,3],
    'greet' : 'hello',
    'age'  : 20
}
print('hello' in user.keys())

False


In [33]:
user = {
    'basket' : [1,2,3],
    'greet' : 'hello',
    'age'  : 20
}
print('hello' in user.values())

True


In [29]:
user = {
    'basket' : [1,2,3],
    'greet' : 'hello',
    'age'  : 20
}
print(user.get('age'))

20


In [22]:
a,b,c, * other, d = [1,2,3,4,5,6,7,8,9]
print(a)
print(b)
print(c)
print(other)
print(d)

1
2
3
[4, 5, 6, 7, 8]
9


In [28]:
dict = {'a':1, 'b' : 'hellloo', 'c': True}
print(dict['c'])

True


In [25]:
dict = {'a':1, 'b' : 2, 'c':7}
print(dict)

{'a': 1, 'b': 2, 'c': 7}


In [20]:
basket = ['a','b','z','v','d']
basket.sort()
basket.reverse()
print(basket[::1])

['z', 'v', 'd', 'b', 'a']


In [ ]:
basket.pop()
print(basket)

[1, 2, 3, 4, 5]


In [17]:
basket = ['a','b','c']
print(basket.index('b'))

1


In [18]:
basket = ['a','b','z','v','d']
basket.sort()
print(basket)

['a', 'b', 'd', 'v', 'z']


In [ ]:
basket = [1,2,3,4,5]
new_list = basket.append(100)
print(basket)
print(new_list)

[1, 2, 3, 4, 5, 100]
None


In [ ]:
amazon_cart = ['notebooks', 'sunglasses', 'toys', 'bicycles']
print(amazon_cart[0:2])

['notebooks', 'sunglasses']


In [ ]:
amazon_cart[0] = 'laptops'
print(amazon_cart)

['laptops', 'sunglasses', 'toys', 'bicycles']


In [ ]:
matrix = [[0,1,2],[3,4,5],[6,7,8]]

print(matrix[0][2])

2


In [ ]:
basket = [1,2,3]
print(len(basket))

3


In [ ]:
username = input('what is your name?')
password = input('what is your password?')

password_length = len(password)
hidden_password = '*' * password_length

print(f'{username}, your password, {hidden_password}, is {password_length} letters long')

what is your name?shirin
what is your password?Shirinrafia*123
shirin, your password, ***************, is 15 letters long


# **HighIQ Python Trainer Notebook** *(Zero to One)*

In [ ]:
amazon_cart = ['notebooks', 'sunglasses']
print(amazon_cart[0])

notebooks


In [ ]:
birth_year = input('what year you were born?')

age = 2022 - int(birth_year)

print(f'your age is : {age} ')

what year you were born?2000
your age is : 22 


In [ ]:
name = 'shirin'
is_nice = False
is_nice = True

print(bool(0))

False


In [ ]:
selfish = 'me me me'
print(selfish[6]) 


m


In [ ]:
weather = "It's \"kind of\"sunny"
print(weather)

It's "kind of"sunny


In [ ]:
a = str(100)
b = int(a)
c = type(b)
print(c)

<class 'int'>


In [ ]:
print(type(int(str(100))))

<class 'int'>


In [ ]:
print('hey '  +  'shirin')

hey shirin


In [ ]:
print(type('hi shirin'))
long_string = '''
WOW
0  0
----
'''
print(long_string)

<class 'str'>

WOW
0  0
----



In [ ]:
iq = 100
user_age = iq/5

In [ ]:
some_value = 5
some_value = some_value + 6

print(some_value)

11


# [**Click on this Course**](https://www.youtube.com/watch?v=eWRfhZUzrAc&t=2s)

In [ ]:
print(bin(5))

0b101


In [ ]:
input('what is your age?')

what is your age?22


'22'

In [ ]:
print("shirinrafia")

shirinrafia


## Variable And Functions

## Calling Funtions

# Dictionaries

## User Input

## Libraries, Lists, Methods

## Function Arguments

## If Statements 

## Concatenating Strings

## f-strings

## Else and Elif Statements

## Refactoring and Nested If

## Accessing Dictionary Values

## Testing Game

## Variables

## Expressions and Statements 

## Comments

## Data Types

## Operators

## Arithmetic Operators

## Comparison Operators

## Boolean Operators

## Bitwise Operators

## is & in Operators

## Ternary Operator

## Strings

## String Methods

## Escaping Characters

## String Characters & Slicing

## Booleans

## Number Data Types

## Built-in Functions

In [ ]:
print(round(-5.5))

-6


## Enums

In [ ]:
from enum import Enum

class State(Enum):
  INACTIVE = 0
  ACTIVE = 1

print(State.ACTIVE)

State.ACTIVE


## User Input

In [ ]:
print("What is your age? ")
age = input()
print("your age is  "  + age)


What is your age? 
22
your age is  22


## Control Statements

In [ ]:
condition = False

if condition == True:
  print()


## Lists

In [ ]:
dogs = ["Shirin", 1, "Rafia", True]

print("Shirin" in dogs)

True


## Sorting Lists

In [ ]:
from typing import ItemsView
items = ["shirin", "rafia", "Beau", "Quincy"]
itemscopy = items[:]
items.sort(key=str.lower)

print(items)
print(itemscopy)

['Beau', 'Quincy', 'rafia', 'shirin']
['shirin', 'rafia', 'Beau', 'Quincy']


## Tuples

In [ ]:
names = ("rogger", "aadhil")

names[-1]
names.index("rogger")

len(names)

print("rogger" in names)

True


## Dictionaries

In [ ]:
dog = {"name": "Rafia", "age": 8}

print(dog['name'])

Rafia


## Sets

In [ ]:
set1 = {"rafia", "shirin"}
set2 = {"rogger"}

intersect = set1 & set2
print(intersect)

set()


## Functions

In [ ]:
def hello(name):
  print('Hello ' + name)

  hello("shirin")
  hello("thaaka")

## Variable Scope

In [ ]:
  def test():
    age = 8
    print(age)


## Nested Functions 

In [ ]:
def talk(phrase):
  def say(word):
    print(word)

    words = phrase.split(' ')
    for word in words:
      say(word)

      talk('I am going to buy milk')


## Closures

In [ ]:
def counter():
  count = 0

  def increment():
    nonlocal count
    count = count + 1
    return count

    return increment
    increment = counter()

    print(increment()) #1
    print(increment()) #2
    print(increment()) #3


## Objects

In [ ]:
age = 8

print(age.real)

8


## Loops

In [ ]:
condition = True
while condition == True:
  print("The condition is True")
  condition = False

The condition is True


## Break and Continue

In [ ]:
items = [1, 2, 3, 4]
for item in items:
  if item == 2:
    continue
    print(item)

## Classes

In [ ]:
class Dog:
  def bark(self):
    print("woof!")

## Modules

In [ ]:
def bark():
  print("woof!")

## Arguments from Command Line

In [ ]:
print("hello")

hello


## Lambda Functions

In [ ]:
lambda num : num * 2

<function __main__.<lambda>(num)>

## Map, Filter, Reduce

In [ ]:
numbers = [1, 2, 3]

def double(a):
  return a * 2

  result = map(double, numbers)

  def fun(variable):
    letters = ['a', 'e', 'i', 'o', 'u']
    if (variable in letters):
        return True
    else:
        return False

## Recursion 

In [ ]:
def factorial(x):


    if x == 1:
        return 1
    else:
        return (x * factorial(x-1))


num = 3
print("The factorial of", num, "is", factorial(num))

The factorial of 3 is 6


# The remaining modules from the Video is Optional if you wish to continue you can continue

In [ ]:
def first(msg):
    print(msg)


first("Hello")

second = first
second("Hello")

Hello
Hello
